# Model Evaluation

This notebook demonstrates how you can run my pre-trained model on unseen test data.

In [ ]:
# My virtual environment is tracked using `pipenv`.
# From the top directory of the project, run:
!pipenv install

In [ ]:
import sys; sys.path.append('..')

from functools import partial
import tqdm, os, json

import pandas as pd
import torch
import torch.nn as nn
import gvpgnn.datasets as datasets
import gvpgnn.models as models
import gvpgnn.paths as paths
import gvpgnn.data_models as dm
import gvpgnn.embeddings as embeddings
import gvpgnn.train_utils as train_utils
import numpy as np
import torch_geometric
from sklearn.metrics import confusion_matrix
from scripts.parser import parser